In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:99% !important; }</style>"))

In [2]:
import pyspark
import pandas as pd
import numpy as np
import matplotlib
import os

In [3]:
hadoop_home = os.environ.get('HADOOP_HOME', None)
spark_home = os.environ.get('PYSPARK_PYTHON', None)
java_home = os.environ.get('JAVA_HOME', None)
print(hadoop_home,'\n',spark_home,'\n',java_home)

None 
 None 
 None


In [4]:
sc = pyspark.SparkContext.getOrCreate() 
sc

23/07/01 16:15:19 WARN Utils: Your hostname, Alashmony-Lenovo-Z51-70 resolves to a loopback address: 127.0.1.1; using 192.168.1.182 instead (on interface wlp3s0)
23/07/01 16:15:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/01 16:15:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

In [5]:
spark = pyspark.sql.SparkSession(sc)
spark

# 01 DataFrame details

## Intro to data cleaning with Apache Spark

1. Intro to data cleaning with Apache Spark

Welcome to Data Cleaning in Apache Spark with Python. My name is Mike Metzger, I am a Data Engineering Consultant, and I will be your instructor for this course. We will cover what data cleaning is, why it's important, and how to implement it with Spark and Python. Let's get started!

2. What is Data Cleaning?

In this course, we'll define "data cleaning" as preparing raw data for use in processing pipelines. We'll discuss what a pipeline is later on, but for now, it's sufficient to say that data cleaning is a necessary part of any production data system. If your data isn't "clean", it's not trustworthy and could cause problems later on. There are many tasks that could fall under the data cleaning umbrella. A few of these include reformatting or replacing text; performing calculations based on the data; and removing garbage or incomplete data.

3. Why perform data cleaning with Spark?

Most data cleaning systems have two big problems: optimizing performance and organizing the flow of data. A typical programming language (such as Perl, C++, or even standard SQL) may be able to clean data when you have small quantities of data. But consider what happens when you have millions or even billions of pieces of data. Those languages wouldn't be able to process that amount of information in a timely manner. Spark lets you scale your data processing capacity as your requirements evolve. Beyond the performance issues, dealing with large quantities of data requires a process, or pipeline of steps. Spark allows management of many complex tasks within a single framework.

4. Data cleaning example

Here's an example of cleaning a small data set. We're given a table of names, age in years, and a city. Our requirements are for a DataFrame with first and last name in separate columns, the age in months, and which state the city is in. We also want to remove any rows where the data is out of the ordinary. Using Spark transformations, we can create a DataFrame with these properties and continue processing afterwards.

5. Spark Schemas

A primary function of data cleaning is to verify all data is in the expected format. Spark provides a built-in ability to validate datasets with schemas. You may have used schemas before with databases or XML; Spark is similar. A schema defines and validates the number and types of columns for a given DataFrame. A schema can contain many different types of fields - integers, floats, dates, strings, and even arrays or mapping structures. A defined schema allows Spark to filter out data that doesn't conform during read, ensuring expected correctness. In addition, schemas also have performance benefits. Normally a data import will try to infer a schema on read - this requires reading the data twice. Defining a schema limits this to a single read operation.

6. Example Spark Schema

Here is an example schema to the import data from our previous example. First we'll import the pyspark.sql.types library. Next we define the actual StructType list of StructFields, containing an entry for each field in the data. Each StructField consists of a field name, dataType, and whether the data can be null. Once our schema is defined, we can add it into our spark.read.format.load call and process it against our data. The load() method takes two arguments - the filename and a schema. This is where we apply our schema to the data being loaded.

7. Let's practice!

We've gone over a lot of information regarding data cleaning and the importance of dataframe schemas. Let's put that information to use and practice!

### Data cleaning review
There are many benefits for using Spark for data cleaning.

Which of the following is NOT a benefit?

- Spark offers high performance.

- **Spark can only handle thousands of records.**

- Spark allows orderly data flows.

- Spark can use strictly defined schemas while ingesting data.

### Defining a schema
Creating a defined schema helps with data quality and import performance. As mentioned during the lesson, we'll create a simple schema to read in the following columns:

Name
Age
City
The `Name` and `City` columns are `StringType()` and the `Age` column is an `IntegerType()`.

**Instructions**

- Import * from the `pyspark.sql.types` library.
- Define a new schema using the `StructType` method.
- Define a `StructField` for `name`, `age`, and `city`. Each field should correspond to the correct datatype and not be nullable.

In [6]:
# Import the pyspark.sql.types library
from pyspark.sql.types import *

# Define a new schema using the StructType method
people_schema = StructType([
  # Define a StructField for each field
  StructField('name', StringType(), False),
  StructField('age', IntegerType(), False),
  StructField('city', StringType(), False)
])

## Immutability and lazy processing

1. Immutability and Lazy Processing

Welcome back! We've had a quick discussion about data cleaning, data types and schemas. Let's move on to some further Spark concepts - Immutability and Lazy Processing.

2. Variable review

Normally in Python, and most other languages, variables are fully mutable. The values can be changed at any given time, assuming the scope of the variable is valid. While very flexible, this does present problems anytime there are multiple concurrent components trying to modify the same data. Most languages work around these issues using constructs like mutexes, semaphores, etc. This can add complexity, especially with non-trivial programs.

3. Immutability

Unlike typical Python variables, Spark Data Frames are immutable. While not strictly required, immutability is often a component of functional programming. We won't go into everything that implies here, but understand that Spark is designed to use immutable objects. Practically, this means Spark Data Frames are defined once and are not modifiable after initialization. If the variable name is reused, the original data is removed (assuming it's not in use elsewhere) and the variable name is reassigned to the new data. While this seems inefficient, it actually allows Spark to share data between all cluster components. It can do so without worry about concurrent data objects.

4. Immutability Example

This is a quick example of the immutability of data frames in Spark. It's OK if you don't understand the actual code, this example is more about the concepts of what happens. First, we create a data frame from a CSV file called voterdata.csv. This creates a new data frame definition and assigns it to the variable name voter_df. Once created, we want to do two further operations. The first is to create a fullyear column by using a 2-digit year present in the data set and adding 2000 to each entry. This does not actually change the data frame at all. It copies the original definition, adds the transformation, and assigns it to the voter_df variable name. Our second operation is similar - now we want to drop the original year column from the data frame. Again, this copies the definition, adds a transformation and reassigns the variable name to this new object. The original objects are destroyed. Please note that the original year column is now permanently gone from this instance, though not from the underlying data (ie, you could simply reload it to a new dataframe if desired).

5. Lazy Processing

You may be wondering how Spark does this so quickly, especially on large data sets. Spark can do this because of something called lazy processing. Lazy processing in Spark is the idea that very little actually happens until an action is performed. In our previous example, we read a CSV file, added a new column, and deleted another. The trick is that no data was actually read / added / modified, we only updated the instructions (aka, Transformations) for what we wanted Spark to do. This functionality allows Spark to perform the most efficient set of operations to get the desired result. The code example is the same as the previous slide, but with the added count() method call. This classifies as an action in Spark and will process all the transformation operations.

6. Let's practice!

These concepts can be a little tricky to grasp without some examples. Let's practice these ideas in the coming exercises.

### Immutability review
You’ve just seen that immutability and lazy processing are fundamental concepts in the way Spark handles data. But why would Spark use immutable data frames to begin with?


- To add complexity to your Spark tasks.

- **To efficiently handle data throughout the cluster.**

- To easily modify variable values as needed.

- To conserve storage space.

## Using lazy processing
Lazy processing operations will usually return in about the same amount of time regardless of the actual quantity of data. Remember that this is due to Spark not performing any transformations until an action is requested.

For this exercise, we'll be defining a Data Frame (`aa_dfw_df`) and add a couple transformations. Note the amount of time required for the transformations to complete when defined vs when the data is actually queried. These differences may be short, but they will be noticeable. When working with a full Spark cluster with larger quantities of data the difference will be more apparent.

**Instructions**

- Load the Data Frame.
- Add the transformation for `F.lower()` to the Destination Airport column.
- Drop the Destination Airport column from the Data Frame `aa_dfw_df`. Note the time for these operations to complete.
- Show the Data Frame, noting the time difference for this action to complete.

In [7]:
%timeit
# Load the CSV file
aa_dfw_df = spark.read.format('csv').options(Header=True).load('AA_DFW_2017_Departures_Short.csv.gz')


In [8]:
from pyspark.sql import functions as F

In [9]:
%timeit
# Add the airport column using the F.lower() method
aa_dfw_df = aa_dfw_df.withColumn('airport', F.lower(aa_dfw_df['Destination Airport']))

In [10]:
%timeit
# Drop the Destination Airport column
aa_dfw_df = aa_dfw_df.drop(aa_dfw_df['Destination Airport'])

In [11]:
%timeit
# Show the DataFrame
aa_dfw_df.show()

+-----------------+-------------+-----------------------------+-------+
|Date (MM/DD/YYYY)|Flight Number|Actual elapsed time (Minutes)|airport|
+-----------------+-------------+-----------------------------+-------+
|       01/01/2017|         0005|                          537|    hnl|
|       01/01/2017|         0007|                          498|    ogg|
|       01/01/2017|         0037|                          241|    sfo|
|       01/01/2017|         0043|                          134|    dtw|
|       01/01/2017|         0051|                           88|    stl|
|       01/01/2017|         0060|                          149|    mia|
|       01/01/2017|         0071|                          203|    lax|
|       01/01/2017|         0074|                           76|    mem|
|       01/01/2017|         0081|                          123|    den|
|       01/01/2017|         0089|                          161|    slc|
|       01/01/2017|         0096|                           84| 

## Understanding Parquet

1. Understanding Parquet

Welcome back! As we've seen, Spark can read in text and CSV files. While this gives us access to many data sources, it's not always the most convenient format to work with. Let's take a look at a few problems with CSV files.

2. Difficulties with CSV files

Some common issues with CSV files include: The schema is not defined: there are no data types included, nor column names (beyond a header row). Using content containing a comma (or another delimiter) requires escaping. Using the escape character within content requires even further escaping. The available encoding formats are limited depending on the language used.

3. Spark and CSV files

In addition to the issues with CSV files in general, Spark has some specific problems processing CSV data. CSV files are quite slow to import and parse. The files cannot be shared between workers during the import process. If no schema is defined, all data must be read before a schema can be inferred. Spark has feature known as predicate pushdown. Basically, this is the idea of ordering tasks to do the least amount of work. Filtering data prior to processing is one of the primary optimizations of predicate pushdown. This drastically reduces the amount of information that must be processed in large data sets. Unfortunately, you cannot filter the CSV data via predicate pushdown. Finally, Spark processes are often multi-step and may utilize an intermediate file representation. These representations allow data to be used later without regenerating the data from source. Using CSV would instead require a significant amount of extra work defining schemas, encoding formats, etc.

4. The Parquet Format

Parquet is a compressed columnar data format developed for use in any Hadoop based system. This includes Spark, Hadoop, Apache Impala, and so forth. The Parquet format is structured with data accessible in chunks, allowing efficient read / write operations without processing the entire file. This structured format supports Spark's predicate pushdown functionality, providing significant performance improvement. Finally, Parquet files automatically include schema information and handle data encoding. This is perfect for intermediary or on-disk representation of processed data. Note that Parquet files are a binary file format and can only be used with the proper tools. This is in contrast to CSV files which can be edited with any text editor.

5. Working with Parquet

Interacting with Parquet files is very straightforward. To read a parquet file into a Data Frame, you have two options. The first is using the `spark.read.format` method we've seen previously. The Data Frame, df=spark.read.format('parquet').load('filename.parquet') The second option is the shortcut version: The Data Frame, df=spark.read.parquet('filename.parquet') Typically, the shortcut version is the easiest to use but you can use them interchangeably. Writing parquet files is similar, using either: df.write.format('parquet').save('filename.parquet') or df.write.parquet('filename.parquet') The long-form versions of each permit extra option flags, such as when overwriting an existing parquet file.

6. Parquet and SQL

Parquet files have various uses within Spark. We've discussed using them as an intermediate data format, but they also are perfect for performing SQL operations. To perform a SQL query against a Parquet file, we first need to create a Data Frame via the spark.read.parquet method. Once we have the Data Frame, we can use the createOrReplaceTempView() method to add an alias of the Parquet data as a SQL table. Finally, we run our query using normal SQL syntax and the spark.sql method. In this case, we're looking for all flights with a duration under 100 minutes. Because we're using Parquet as the backing store, we get all the performance benefits we've discussed previously (primarily defined schemas and the available use of predicate pushdown).

7. Let's Practice!

You've seen a bit about what a Parquet file is and why we'd want to use them. Now, let's practice working with Parquet files.

## Saving a DataFrame in Parquet format
When working with Spark, you'll often start with CSV, JSON, or other data sources. This provides a lot of flexibility for the types of data to load, but it is not an optimal format for Spark. The Parquet format is a columnar data store, allowing Spark to use predicate pushdown. This means Spark will only process the data necessary to complete the operations you define versus reading the entire dataset. This gives Spark more flexibility in accessing the data and often drastically improves performance on large datasets.

In this exercise, we're going to practice creating a new Parquet file and then process some data from it.

The spark object and the `df1` and `df2` DataFrames have been setup for you.

**Instructions**

- View the row count of `df1` and `df2`.
- Combine `df1` and `df2` in a new DataFrame named `df3` with the union method.
- Save `df3` to a parquet file named `AA_DFW_ALL.parquet`.
- Read the `AA_DFW_ALL.parquet` file and show the count.

In [12]:
df1 = spark.read.csv('AA_DFW_2014_Departures_Short.csv.gz', header= True)
df2 = spark.read.csv('AA_DFW_2015_Departures_Short.csv.gz', header= True)

### If Spark was not able to save as Parquet format, change the winutil version used as Hadoop Home.
Mostly, you will need to upgrade WinUtil to match the same Spark version, in my case, Spark 3.4 needs to match Hadoop 3.4. [This repo](https://github.com/kontext-tech/winutils.git) has newer version but not fully trusted. For me, it solved the issue.

In [13]:
hadoop_home = os.environ.get('HADOOP_HOME', None)
hadoop_home

In [14]:
# View the row count of df1 and df2
print("df1 Count: %d" % df1.count())
print("df2 Count: %d" % df2.count())

# Combine the DataFrames into one
df3 = df1.union(df2)

#rename column to be used in the next cell
df3 = df3.withColumnRenamed('Actual elapsed time (Minutes)', 'flight_duration')

# Save the df3 DataFrame in Parquet format
df3.write.parquet('AA_DFW_ALL.parquet', mode='overwrite')

# Read the Parquet file into a new DataFrame and run a count
print(spark.read.parquet('AA_DFW_ALL.parquet').count())

df1 Count: 157198
df2 Count: 146558


303756


In [15]:
df1.show()

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2014|         0005|                HNL|                          519|
|       01/01/2014|         0007|                OGG|                          505|
|       01/01/2014|         0035|                SLC|                          174|
|       01/01/2014|         0043|                DTW|                          153|
|       01/01/2014|         0052|                PIT|                          137|
|       01/01/2014|         0058|                SAN|                          174|
|       01/01/2014|         0060|                MIA|                          155|
|       01/01/2014|         0064|                JFK|                          185|
|       01/01/2014|         0090|                ORD|                       

In [16]:
df2.show()

+-----------------+-------------+-------------------+-----------------------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|Actual elapsed time (Minutes)|
+-----------------+-------------+-------------------+-----------------------------+
|       01/01/2015|         0005|                HNL|                          526|
|       01/01/2015|         0007|                OGG|                          517|
|       01/01/2015|         0023|                SFO|                          233|
|       01/01/2015|         0027|                LAS|                          165|
|       01/01/2015|         0029|                ONT|                            0|
|       01/01/2015|         0035|                HDN|                          178|
|       01/01/2015|         0037|                SAN|                          187|
|       01/01/2015|         0043|                DTW|                            0|
|       01/01/2015|         0049|                SAN|                       

In [17]:
df3.show()

+-----------------+-------------+-------------------+---------------+
|Date (MM/DD/YYYY)|Flight Number|Destination Airport|flight_duration|
+-----------------+-------------+-------------------+---------------+
|       01/01/2014|         0005|                HNL|            519|
|       01/01/2014|         0007|                OGG|            505|
|       01/01/2014|         0035|                SLC|            174|
|       01/01/2014|         0043|                DTW|            153|
|       01/01/2014|         0052|                PIT|            137|
|       01/01/2014|         0058|                SAN|            174|
|       01/01/2014|         0060|                MIA|            155|
|       01/01/2014|         0064|                JFK|            185|
|       01/01/2014|         0090|                ORD|            126|
|       01/01/2014|         0096|                STL|             91|
|       01/01/2014|         0099|                SNA|            182|
|       01/01/2014| 

### SQL and Parquet
Parquet files are perfect as a backing data store for SQL queries in Spark. While it is possible to run the same queries directly via Spark's Python functions, sometimes it's easier to run SQL queries alongside the Python options.

For this example, we're going to read in the Parquet file we created in the last exercise and register it as a SQL table. Once registered, we'll run a quick query against the table (aka, the Parquet file).

The spark object and the `AA_DFW_ALL.parquet` file are available for you automatically.

**Instructions**

- Import the `AA_DFW_ALL.parquet` file into flights_df.
- Use the `createOrReplaceTempView` method to alias the flights table.
- Run a Spark SQL query against the flights table.

In [18]:
# Read the Parquet file into flights_df
flights_df = spark.read.parquet('AA_DFW_ALL.parquet')

# Register the temp table
flights_df.createOrReplaceTempView('flights')

# Run a SQL query of the average flight duration
avg_duration = spark.sql('SELECT avg(flight_duration) from flights').collect()[0]
print('The average flight time is: %d' % avg_duration)

The average flight time is: 143


# Chapter 2: Manipulating DataFrames in the real world

## Manipulating DataFrames in the real world
1. DataFrame column operations

Welcome back! In the first chapter, we've spent some time discussing the basics of Spark data and file handling. Let's now take a look at how to use Spark column operations to clean data.

2. DataFrame refresher

Before we discuss manipulating DataFrames in depth, let's talk about some of their features. DataFrames are made up of rows & columns and are generally analogous to a database table. DataFrames are immutable: any change to the structure or content of the data creates a new DataFrame. DataFrames are modified through the use of transformations. An example is The .filter() command to only return rows where the name starts with the letter 'M'. Another operation is .select(), in this case returning only the name and position fields.

3. Common DataFrame transformations

There are many different transformations for use on a DataFrame. They vary depending on what you'd like to do. Some common transformations include: The .filter() clause, which includes only rows that satisfy the requirements defined in the argument. This is analogous to the WHERE clause in SQL. Spark includes a .where() alias which you can use in place of .filter() if desired. This call returns only rows where the vote occurred after 1/1/2019. Another common option is the .select() method which returns the columns requested from the DataFrame. The .withColumn() method creates a new column in the DataFrame. The first argument is the name of the column, and the second is the command(s) to create it. In this case, we create a column called 'year' with just the year information. We also can use the .drop() method to remove a column from a DataFrame.

4. Filtering data

Among the most common operations used when cleaning a DataFrame, filtering lets us use only the data matching our desired result. We can use .filter() for many tasks, such as: Removing null values. Removing odd entries, anything that doesn't fit our desired format. We can also split a DataFrame containing combined data (such as a syslog file). As mentioned previously, use the .filter() method to return only rows that meet the specified criteria. The .contains() function takes a string argument that the column must have to return true. You can negate these results using the tile (~) character.

5. Column string transformations

Some of the most common operations used in data cleaning are modifying and converting strings. You will typically apply these to each column as a transformation. Many of these functions are in the pyspark.sql.functions library. For brevity, we'll import it as the alias 'F'. We use the .withColumn() function to create a new column called "upper" using pyspark.sql.functions.upper() on the name column. The "upper" column will contain uppercase versions of all names. We can create intermediary columns that are only for processing. This is useful to clarify complex transformations requiring multiple steps. In this instance, we call the .split() function with the name of the column and the space character to split on. This returns a list of words in a column called splits. A very common operation is converting string data to a different type, such as converting a string column to an integer. We use the .cast() function to perform the conversion to an IntegerType().

6. ArrayType() column functions

While performing data cleaning with Spark, you may need to interact with ArrayType() columns. These are analogous to lists in normal python environments. One function we will use is .size(), which returns the number of items present in the specified ArrayType() argument. Another commonly used function for ArrayTypes is .getItem(). It takes an index argument and returns the item present at that index in the list column. Spark has many more transformations and utility functions available. When using Spark in production, make sure to reference the documentation for available options.

7. Let's practice!

We've discussed some of the common operations used on Spark DataFrame columns. Let's practice some of these now.

### Filtering column content with Python
You've looked at using various operations on DataFrame columns - now you can modify a real dataset. The DataFrame `voter_df` contains information regarding the voters on the Dallas City Council from the past few years. This truncated DataFrame contains the date of the vote being cast and the name and position of the voter. Your manager has asked you to clean this data so it can later be integrated into some desired reports. The primary task is to remove any null entries or odd characters and return a specific set of voters where you can validate their information.

This is often one of the first steps in data cleaning - removing anything that is obviously outside the format. For this dataset, make sure to look at the original data and see what looks out of place for the `VOTER_NAME` column.

The `pyspark.sql.functions` library is already imported under the alias `F`.

**Instructions**

- Show the distinct `VOTER_NAME` entries.
- Filter `voter_df` where the `VOTER_NAME` is 1-20 characters in length.
- Filter out `voter_df` where the `VOTER_NAME` contains an `_`.
- Show the distinct `VOTER_NAME` entries again.

In [52]:
voter_df = spark.read.csv('DallasCouncilVoters.csv.gz', header=True,inferSchema=True)

In [53]:
# Show the distinct VOTER_NAME entries
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|VOTER_NAME                                                                                                                                                                                                                                                                                                                                                                                                                 |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [54]:

# Filter voter_df where the VOTER_NAME is 1-20 characters in length
voter_df = voter_df.where('length(VOTER_NAME) > 0 and length(VOTER_NAME) < 20')

# Filter out voter_df where the VOTER_NAME contains an underscore
voter_df = voter_df.filter(~ F.col('VOTER_NAME').contains('_'))

# Show the distinct VOTER_NAME entries again
voter_df.select('VOTER_NAME').distinct().show(40, truncate=False)

+-------------------+
|VOTER_NAME         |
+-------------------+
|Tennell Atkins     |
|Scott Griggs       |
|Scott  Griggs      |
|Sandy Greyson      |
|Michael S. Rawlings|
|Kevin Felder       |
|Adam Medrano       |
|Casey  Thomas      |
|Mark  Clayton      |
|Casey Thomas       |
|Sandy  Greyson     |
|Mark Clayton       |
|Jennifer S.  Gates |
|Tiffinni A. Young  |
|B. Adam  McGough   |
|Omar Narvaez       |
|Philip T. Kingston |
|Rickey D. Callahan |
|Dwaine R. Caraway  |
|Philip T.  Kingston|
|Jennifer S. Gates  |
|Lee M. Kleinman    |
|Monica R. Alonzo   |
|Rickey D.  Callahan|
|Carolyn King Arnold|
|Erik Wilson        |
|Lee Kleinman       |
+-------------------+



### Filtering Question #1
Consider the following Data Frame called `users_df`:

|ID| Name| Age|	State|
|---|---|---|---|
|140|	George| L|	47|	Iowa|
|3260|	Mary| R|	34|	Vermont|
|18502|	null|	68|	Ohio|
|999|	Rick| W|	23|	California|

If you wanted to return only the entries without nulls, which of following options would **not** work?

**Answer the question**
**Possible Answers**

- users_df = users_df.filter(users_df.Name.isNotNull())

- **users_df = users_df.where(users_df.ID == 18502)**

- users_df = users_df.where(~ (users_df.ID == 18502) )

- users_df = users_df.filter(~ col('Name').isNull())


### Modifying DataFrame columns
Previously, you filtered out any rows that didn't conform to something generally resembling a name. Now based on your earlier work, your manager has asked you to create two new columns - `first_name` and `last_name`. She asks you to split the `VOTER_NAME` column into words on any space character. You'll treat the last word as the `last_name`, and all other words as the `first_name`. You'll be using some new functions in this exercise including `.split()`, `.size()`, and `.getItem()`. The `.getItem(index)` takes an integer value to return the appropriately numbered item in the column. The functions `.split()` and `.size()` are in the `pyspark.sql.functions` library.

Please note that these operations are always somewhat specific to the use case. Having your data conform to a format often matters more than the specific details of the format. Rarely is a data cleaning task meant just for one person - matching a defined format allows for easier sharing of the data later (ie, Paul doesn't need to worry about names - Mary already cleaned the dataset).

The filtered voter DataFrame from your previous exercise is available as `voter_df`. The `pyspark.sql.functions` library is available under the alias `F`.

**Instructions**

- Add a new column called splits holding the list of possible names.
- Use the `getItem()` method and create a new column called `first_name`.
- Get the last entry of the splits list and create a column called `last_name`.
- Drop the splits column and show the new `voter_df`.

In [55]:
# No need to import again, just aliassing it
F = pyspark.sql.functions

In [56]:
voter_df.show(5)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
+----------+-------------+-------------------+
only showing top 5 rows



In [57]:
# Add a new column called splits separated on whitespace
voter_df = voter_df.withColumn('splits', F.split(voter_df.VOTER_NAME, '\s+'))

# Create a new column called first_name based on the first item in splits
voter_df = voter_df.withColumn('first_name', voter_df.splits.getItem(0))

# Get the last entry of the splits list and create a column called last_name
voter_df = voter_df.withColumn('last_name', voter_df.splits.getItem(F.size('splits')-1))

# Drop the splits column
#voter_df = voter_df.drop('splits') #Hashes as we will need it later

# Show the voter_df DataFrame
voter_df.show()

+----------+-------------+-------------------+--------------------+----------+---------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|
+----------+-------------+-------------------+--------------------+----------+---------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas|
|02/08/2017|Councilmember|Carolyn King Arnold|[Carolyn, King, A...|   Carolyn|   Arnold|
|02/08/2017|Councilmember|       Scott Griggs|     [Scott, Griggs]|     Scott|   Griggs|
|02/08/2017|Councilmember|   B. Adam  McGough| [B., Adam, McGough]|        B.|  McGough|
|02/08/2017|Councilme

/home/alashmony/.local/lib/python3.10/site-packages/pyspark/sql/column.py:458: FutureWarning: A column as 'key' in getItem is deprecated as of Spark 3.0, and will not be supported in the future release. Use `column[key]` or `column.key` syntax instead.
  warnings.warn(


## Conditional DataFrame column operations

1. Conditional DataFrame column operations

We've looked at some of the power available when using Spark's functions to filter and modify our Data Frames. Let's spend some time with some more advanced options.

2. Conditional clauses

The DataFrame transformations we've covered thus far are blanket transformations, meaning they're applied regardless of the data. Often you'll want to conditionally change some aspect of the contents. Spark provides some built in conditional clauses which act similar to an if / then / else statement in a traditional programming environment. While it is possible to perform a traditional if / then / else style statement in Spark, it can lead to serious performance degradation as each row of a DataFrame would be evaluated independently. Using the optimized, built-in conditionals alleviates this. There are two components to the conditional clauses: .when(), and the optional .otherwise(). Let's look at them in more depth.

3. Conditional example

The .when() clause is a method available from the pyspark.sql.functions library that is looking for two components: the if condition, and what to do if it evaluates to true. This is best seen from an example. Consider a DataFrame with the Name and Age columns. We can actually add an extra argument to our .select() method using the .when() clause. We select df.Name and df.Age as usual. For the third argument, we'll define a when conditional. If the Age column is 18 or up, we'll add the string "Adult". If the clause doesn't match, nothing is returned. Note that our returned DataFrame contains an unnamed column we didn't define using .withColumn(). The .select() function can create columns dynamically based on the arguments provided. Let's look at some more examples.

4. Another example

You can chain multiple when statements together, similar to an if / else if structure. In this case, we define two .when() clauses and return Adult or Minor based on the Age column. You can chain as many when clauses together as required.

5. Otherwise

In addition to .when() is the otherwise() clause. .otherwise() is analogous to the else statement. It takes a single argument, which is what to return, in case the when clause or clauses do not evaluate as True. In this example, we return "Adult" when the Age column is 18 or higher. Otherwise, we return "Minor". The resulting DataFrame is the same, but the method is different. While you can have multiple .when() statements chained together, you can only have a single .otherwise() per .when() chain.

6. Let's practice!

Let's try a couple examples of using .when() and .otherwise() to modify some DataFrames!

### when() example
The `when()` clause lets you conditionally modify a Data Frame based on its content. You'll want to modify our `voter_df` DataFrame to add a random number to any voting member that is defined as a "Councilmember".

The `voter_df` DataFrame is defined and available to you. The `pyspark.sql.functions` library is available as `F`. You can use `F.rand()` to generate the random value.

**Instructions**

- Add a column to `voter_df` named `random_val` with the results of the `F.rand()` method for any voter with the title Councilmember.
- Show some of the DataFrame rows, noting whether the `.when()` clause worked.

In [58]:
# Add a column to voter_df for any voter with the title **Councilmember**
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand()))

# Show some of the DataFrame rows, noting whether the when clause worked
voter_df.show()

+----------+-------------+-------------------+--------------------+----------+---------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|         random_val|
+----------+-------------+-------------------+--------------------+----------+---------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|0.10948335830268863|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|  0.720920387946472|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|               null|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|0.14653774208919867|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas| 0.4376974349574557|
|02/08/2017|Councilmember|Carolyn King Arnold|[Carolyn, King, A...|   Carolyn|   Arnold| 0.9303114670481522|
|02/08/2017|Council

### When / Otherwise
This requirement is similar to the last, but now you want to add multiple values based on the voter's position. Modify your `voter_df` DataFrame to add a random number to any voting member that is defined as a Councilmember. Use 2 for the Mayor and 0 for anything other position.

The `voter_df` Data Frame is defined and available to you. The `pyspark.sql.functions` library is available as `F`. You can use `F.rand()` to generate the random value.

**Instructions**

- Add a column to `voter_df` named `random_val` with the results of the `F.rand()` method for any voter with the title Councilmember. Set `random_val` to 2 for the Mayor. Set any other title to the value 0.
- Show some of the Data Frame rows, noting whether the clauses worked.
- Use the `.filter clause` to find 0 in `random_val`.

In [59]:
# Add a column to voter_df for a voter based on their position
voter_df = voter_df.withColumn('random_val',
                               F.when(voter_df.TITLE == 'Councilmember', F.rand())
                               .when(voter_df.TITLE == 'Mayor', 2)
                               .otherwise(0))

# Show some of the DataFrame rows
voter_df.show()

# Use the .filter() clause with random_val
voter_df.filter(voter_df.random_val == 0).show()

+----------+-------------+-------------------+--------------------+----------+---------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|         random_val|
+----------+-------------+-------------------+--------------------+----------+---------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|0.40103231355053004|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|  0.292098313673427|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|                2.0|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|0.18192386578773923|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Thomas| 0.9851254350409162|
|02/08/2017|Councilmember|Carolyn King Arnold|[Carolyn, King, A...|   Carolyn|   Arnold|   0.63992939289795|
|02/08/2017|Council

## User defined functions
1. User defined functions

We've looked at the built-in functions in Spark and have had great results using these. But let's consider what you would do if you needed to apply some custom logic to your data cleaning processes.

2. Defined...

A user defined function, or UDF, is a Python method that the user writes to perform a specific bit of logic. Once written, the method is called via the pyspark.sql.functions.udf() method. The result is stored as a variable and can be called as a normal Spark function. Let's look at a couple examples.

3. Reverse string UDF

Here is a fairly trivial example to illustrate how a UDF is defined. First, we define a python function. We'll call our function, reverseString(), with an argument called mystr. We'll use some python shorthand to reverse the string and return it. Don't worry about understanding how the return statement works, only that it will reverse the lettering of whatever is fed into it (ie, "help" becomes "pleh"). The next step is to wrap the function and store it in a variable for later use. We'll use the pyspark.sql.functions.udf() method. It takes two arguments - the name of the method you just defined, and the Spark data type it will return. This can be any of the options in pyspark.sql.types, and can even be a more complex type, including a fully defined schema object. Most often, you'll return either a simple object type, or perhaps an ArrayType. We'll call udf with our new method name, and use the StringType(), then store this as udfReverseString(). Finally, we use our new UDF to add a column to the user_df DataFrame within the .withColumn() method. Note that we pass the column we're interested in as the argument to udfReverseString(). The udf function is called for each row of the Data Frame. Under the hood, the udf function takes the value stored for the specified column (per row) and passes it to the python method. The result is fed back to the resulting DataFrame.

4. Argument-less example

Another quick example is using a function that does not require an argument. We're defining our sortingCap() function to return one of the letters 'G', 'H', 'R', or 'S' at random. We still create our udf wrapped function, and define the return type as StringType(). The primary difference is calling the function, this time without passing in an argument as it is not required.

5. Let's practice!

As always, the best way to learn is practice - let's create some user defined functions!

### Understanding user defined functions
When creating a new user defined function, which is not a possible value for the second argument?

**Possible Answers**

- ArrayType(IntegerType())

- IntegerType()

- LongType()

- **udf()**

- StringType()

### Using user defined functions in Spark
You've seen some of the power behind Spark's built-in string functions when it comes to manipulating DataFrames. However, once you reach a certain point, it becomes difficult to process the data in a without creating a rat's nest of function calls. Here's one place where you can use User Defined Functions to manipulate our DataFrames.

For this exercise, we'll use our voter_df DataFrame, but you're going to replace the `first_name` column with the first and middle names.

The `pyspark.sql.functions` library is available under the alias `F`. The classes from `pyspark.sql.types` are already imported.

**Instructions**

- Edit the `getFirstAndMiddle()` function to return a space separated string of names, except the last entry in the names list.
- Define the function as a user-defined function. It should return a string type.
- Create a new column on `voter_df` called `first_and_middle_name` using your UDF.

In [60]:
def getFirstAndMiddle(names):
  # Return a space separated string of names
  return ' '.join(names[:-1])

# Define the method as a UDF
udfFirstAndMiddle = F.udf(getFirstAndMiddle, StringType())

# Create a new column using your UDF
voter_df = voter_df.withColumn('first_and_middle_name', udfFirstAndMiddle(voter_df.splits))

# Show the DataFrame
voter_df.show()

+----------+-------------+-------------------+--------------------+----------+---------+-------------------+---------------------+
|      DATE|        TITLE|         VOTER_NAME|              splits|first_name|last_name|         random_val|first_and_middle_name|
+----------+-------------+-------------------+--------------------+----------+---------+-------------------+---------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|[Jennifer, S., Ga...|  Jennifer|    Gates|0.40103231355053004|          Jennifer S.|
|02/08/2017|Councilmember| Philip T. Kingston|[Philip, T., King...|    Philip| Kingston|  0.292098313673427|            Philip T.|
|02/08/2017|        Mayor|Michael S. Rawlings|[Michael, S., Raw...|   Michael| Rawlings|                2.0|           Michael S.|
|02/08/2017|Councilmember|       Adam Medrano|     [Adam, Medrano]|      Adam|  Medrano|0.18192386578773923|                 Adam|
|02/08/2017|Councilmember|       Casey Thomas|     [Casey, Thomas]|     Casey|   Th

## Partitioning and lazy processing
1. Partitioning and lazy processing

Welcome back to our discussion about modifying and cleaning DataFrames. We've discussed various transformations and methods to modify our data, but we haven't covered much about how Spark actually processes the data. Let's look at that now.

2. Partitioning

Spark breaks DataFrames into partitions, or chunks of data. These partitions can be automatically defined, enlarged, shrunk, and can differ greatly based on the type of Spark cluster being used. The size of the partition does vary, but generally try to keep your partition sizes equal. We'll discuss more about optimizing partitioning and cluster details later on. For now, let's assume that each partition is handled independently. This is part of what provides the performance levels and horizontal scaling ability in Spark. If a Spark node doesn't need to compete for resources, nor consult with other Spark nodes for answers, it can reliably schedule the processing for the best performance.

3. Lazy processing

In Spark, any transformation operation is lazy; it's more like a recipe than a command. It defines what should be done to a DataFrame rather than actually doing it. Most operations in Spark are actually transformations, including .withColumn(), .select(), .filter(), and so forth. The set of transformations you define are only executed when you run a Spark action. This includes .count(), .write(), etc - anything that requires the transformations to be run to properly obtain an answer. Spark can reorder transformations for the best performance. Usually this isn't noticeable, but can occasionally cause unexpected behavior, such as IDs not being added until after other transformations have completed. This doesn't actually cause a problem but the data can look unusual if you don't know what to expect.

4. Adding IDs

Relational databases tend to have a field used to identify the row, whether it is for an actual relationship reference, or just for data identification. These IDs are typically an integer that increases in value, is sequential, and most importantly unique. The problem with these IDs is they're not very parallel in nature. Given that the values are given out sequentially, if there are multiple workers, they must all refer to a common source for the next entry. This is OK in a single server environment, but in a distributed platform such as Spark, it creates some undue bottlenecks. Let's take a look at how to generate ID's in Spark.

5. Monotonically increasing IDs

Spark has a built-in function called monotonically_increasing_id(), designed to provide an integer ID that increases in value and is unique. These IDs are not necessarily sequential - there can be gaps, often quite large, between values. Unlike a normal relational ID, Spark's is completely parallel - each partition is allocated up to 8 billion IDs that can be assigned. Notice that the ID fields in the sample table are integers, increasing in value, but are not sequential. It's a little out scope, but the IDs are a 64-bit number effectively split into groups based on the Spark partition. Each group contains 8.4 billion IDs, and there are 2.1 billion possible groups, none of which overlap.

6. Notes

There's a lot of nuance to how partitions and the monotonically increasing ID's work. Remembering that Spark is lazy: it often helps in troubleshooting what can happen. Operations are often out of order - especially if joins are involved. It's best to test your transformations.

7. Let's practice!

We've discussed a lot of detail in this lesson - let's now take a look at a few exercises that will help solidify how everything works.

### Adding an ID Field
When working with data, you sometimes only want to access certain fields and perform various operations. In this case, find all the unique voter names from the DataFrame and add a unique ID number. Remember that Spark IDs are assigned based on the DataFrame partition - as such the ID values may be much greater than the actual number of rows in the DataFrame.

With Spark's lazy processing, the IDs are not actually generated until an action is performed and can be somewhat random depending on the size of the dataset.

The spark session and a Spark DataFrame `df` containing the `DallasCouncilVotes.csv.gz` file are available in your workspace. The `pyspark.sql.functions` library is available under the alias `F`.

**Instructions**

- Select the unique entries from the column `VOTER NAME` and create a new DataFrame called `voter_df`.
- Count the rows in the `voter_df` DataFrame.
- Add a `ROW_ID` column using the appropriate Spark function.
- Show the rows with the 10 highest `ROW_IDs`.

In [67]:
df = spark.read.csv('DallasCouncilVoters.csv.gz',header=True, inferSchema=True)
df.show(5)

+----------+-------------+-------------------+
|      DATE|        TITLE|         VOTER_NAME|
+----------+-------------+-------------------+
|02/08/2017|Councilmember|  Jennifer S. Gates|
|02/08/2017|Councilmember| Philip T. Kingston|
|02/08/2017|        Mayor|Michael S. Rawlings|
|02/08/2017|Councilmember|       Adam Medrano|
|02/08/2017|Councilmember|       Casey Thomas|
+----------+-------------+-------------------+
only showing top 5 rows



In [66]:
# Select all the unique council voters
voter_df = df.select(df["VOTER_NAME"]).distinct()

# Count the rows in voter_df
print("\nThere are %d rows in the voter_df DataFrame.\n" % voter_df.count())

# Add a ROW_ID
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the rows with 10 highest IDs in the set
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)


There are 36 rows in the voter_df DataFrame.

+--------------------+------+
|          VOTER_NAME|ROW_ID|
+--------------------+------+
|        Lee Kleinman|    35|
|  the  final  201...|    34|
|         Erik Wilson|    33|
|  the  final   20...|    32|
| Carolyn King Arnold|    31|
| Rickey D.  Callahan|    30|
|   the   final  2...|    29|
|    Monica R. Alonzo|    28|
|     Lee M. Kleinman|    27|
|   Jennifer S. Gates|    26|
+--------------------+------+
only showing top 10 rows



## IDs with different partitions
You've just completed adding an ID field to a DataFrame. Now, take a look at what happens when you do the same thing on DataFrames containing a different number of partitions.

To check the number of partitions, use the method `.rdd.getNumPartitions()` on a DataFrame.

The spark session and two DataFrames, `voter_df` and `voter_df_single`, are available in your workspace. The instructions will help you discover the difference between the DataFrames. The `pyspark.sql.functions` library is available under the alias `F`.

**Instructions**

- Print the number of partitions on each DataFrame.
- Add a `ROW_ID` field to each DataFrame.
- Show the top 10 IDs in each DataFrame.

In [72]:
voter_df_single = df
voter_df = df
voter_df = voter_df.repartition(5)

In [73]:
# Print the number of partitions in each DataFrame
print("\nThere are %d partitions in the voter_df DataFrame.\n" % voter_df.rdd.getNumPartitions())
print("\nThere are %d partitions in the voter_df_single DataFrame.\n" % voter_df_single.rdd.getNumPartitions())

# Add a ROW_ID field to each DataFrame
voter_df = voter_df.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_single = voter_df_single.withColumn('ROW_ID', F.monotonically_increasing_id())

# Show the top 10 IDs in each DataFrame 
voter_df.orderBy(voter_df.ROW_ID.desc()).show(10)
voter_df_single.orderBy(voter_df_single.ROW_ID.desc()).show(10)


There are 5 partitions in the voter_df DataFrame.


There are 1 partitions in the voter_df_single DataFrame.

+----------+-------------+-----------------+-----------+
|      DATE|        TITLE|       VOTER_NAME|     ROW_ID|
+----------+-------------+-----------------+-----------+
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747292|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747291|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747290|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747289|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747288|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747287|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747286|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747285|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747284|
|06/28/2017|Mayor Pro Tem|Dwaine R. Caraway|34359747283|
+----------+-------------+-----------------+-----------+
only showing top 10 rows

+----------+--------------------+-------------------+------+
|   

## More ID tricks
Once you define a Spark process, you'll likely want to use it many times. Depending on your needs, you may want to start your IDs at a certain value so there isn't overlap with previous runs of the Spark task. This behavior is similar to how IDs would behave in a relational database. You have been given the task to make sure that the IDs output from a monthly Spark task start at the highest value from the previous month.

The spark session and two DataFrames, `voter_df_march` and `voter_df_april`, are available in your workspace. The `pyspark.sql.functions` library is available under the alias `F`.

**Instructions**

- Determine the highest `ROW_ID` in `voter_df_march` and save it in the variable `previous_max_ID`. The statement `.rdd.max()[0]` will get the maximum ID.
- Add a `ROW_ID` column to `voter_df_april` starting at the value of `previous_max_ID`.
- Show the `ROW_ID`'s from both Data Frames and compare.

In [81]:
voter_df_march = df.filter("(DATE > '02/28/2018') and (DATE < '04/01/2018')")
voter_df_march.show()

+----------+--------------------+-------------------+
|      DATE|               TITLE|         VOTER_NAME|
+----------+--------------------+-------------------+
|03/21/2018|       Councilmember|      Scott  Griggs|
|03/21/2018|       Councilmember|   B. Adam  McGough|
|03/21/2018|       Councilmember|    Lee M. Kleinman|
|03/21/2018|       Councilmember|     Sandy  Greyson|
|03/21/2018|       Councilmember| Jennifer S.  Gates|
|03/21/2018|       Councilmember|Philip T.  Kingston|
|03/21/2018|               Mayor|Michael S. Rawlings|
|03/21/2018|Deputy Mayor Pro Tem|       Adam Medrano|
|03/21/2018|       Councilmember|       Casey Thomas|
|03/21/2018|       Mayor Pro Tem|  Dwaine R. Caraway|
|03/21/2018|       Councilmember|Rickey D.  Callahan|
|03/21/2018|       Councilmember|       Omar Narvaez|
|03/21/2018|       Councilmember|       Kevin Felder|
|03/21/2018|       Councilmember|     Tennell Atkins|
|03/21/2018|       Councilmember|      Mark  Clayton|
|03/21/2018|       Councilme

In [82]:
voter_df_april = df.filter("(DATE > '03/31/2018') and (DATE < '05/01/2018')")
voter_df_april.show()

+----------+--------------------+-------------------+
|      DATE|               TITLE|         VOTER_NAME|
+----------+--------------------+-------------------+
|04/25/2018|       Councilmember|     Sandy  Greyson|
|04/25/2018|       Councilmember| Jennifer S.  Gates|
|04/25/2018|       Councilmember|Philip T.  Kingston|
|04/25/2018|               Mayor|Michael S. Rawlings|
|04/25/2018|Deputy Mayor Pro Tem|       Adam Medrano|
|04/25/2018|       Councilmember|       Casey Thomas|
|04/25/2018|       Mayor Pro Tem|  Dwaine R. Caraway|
|04/25/2018|       Councilmember|Rickey D.  Callahan|
|04/25/2018|       Councilmember|       Omar Narvaez|
|04/25/2018|       Councilmember|       Kevin Felder|
|04/25/2018|       Councilmember|     Tennell Atkins|
|04/25/2018|       Councilmember|      Mark  Clayton|
|04/25/2018|       Councilmember|      Scott  Griggs|
|04/25/2018|       Councilmember|   B. Adam  McGough|
|04/25/2018|       Councilmember|    Lee M. Kleinman|
|04/25/2018|       Councilme

In [85]:
voter_df_march = voter_df_march.withColumn('ROW_ID', F.monotonically_increasing_id())
voter_df_march.show()

+----------+--------------------+-------------------+------+
|      DATE|               TITLE|         VOTER_NAME|ROW_ID|
+----------+--------------------+-------------------+------+
|03/21/2018|       Councilmember|      Scott  Griggs|     0|
|03/21/2018|       Councilmember|   B. Adam  McGough|     1|
|03/21/2018|       Councilmember|    Lee M. Kleinman|     2|
|03/21/2018|       Councilmember|     Sandy  Greyson|     3|
|03/21/2018|       Councilmember| Jennifer S.  Gates|     4|
|03/21/2018|       Councilmember|Philip T.  Kingston|     5|
|03/21/2018|               Mayor|Michael S. Rawlings|     6|
|03/21/2018|Deputy Mayor Pro Tem|       Adam Medrano|     7|
|03/21/2018|       Councilmember|       Casey Thomas|     8|
|03/21/2018|       Mayor Pro Tem|  Dwaine R. Caraway|     9|
|03/21/2018|       Councilmember|Rickey D.  Callahan|    10|
|03/21/2018|       Councilmember|       Omar Narvaez|    11|
|03/21/2018|       Councilmember|       Kevin Felder|    12|
|03/21/2018|       Counc

In [86]:
# Determine the highest ROW_ID and save it in previous_max_ID
previous_max_ID = voter_df_march.select('ROW_ID').rdd.max()[0]

# Add a ROW_ID column to voter_df_april starting at the desired value
voter_df_april = voter_df_april.withColumn('ROW_ID', F.monotonically_increasing_id() + previous_max_ID)

# Show the ROW_ID from both DataFrames and compare
voter_df_march.select('ROW_ID').show()
voter_df_april.select('ROW_ID').show()

+------+
|ROW_ID|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows

+------+
|ROW_ID|
+------+
|  1109|
|  1110|
|  1111|
|  1112|
|  1113|
|  1114|
|  1115|
|  1116|
|  1117|
|  1118|
|  1119|
|  1120|
|  1121|
|  1122|
|  1123|
|  1124|
|  1125|
|  1126|
|  1127|
|  1128|
+------+
only showing top 20 rows

